In [1]:
import pandas as pd
import numpy as np
import importlib
import gc
import sys
sys.path.append('./scripts')  
import preprocesamiento_pcp
importlib.reload(preprocesamiento_pcp)

<module 'preprocesamiento_pcp' from 'c:\\Users\\Usuario\\Documents\\Universidad\\austral\\2025\\Lab3\\Lab3-MCD\\notebooks\\entregable\\./scripts\\preprocesamiento_pcp.py'>

In [2]:
ESTOY_EN_KAGGLE = True

In [3]:
df = pd.read_csv("./datasets/periodo_x_cliente_xproducto_con_target.csv", sep=',', encoding='utf-8')
df

,product_id,customer_id,periodo,nacimiento_producto,muerte_producto,nacimiento_cliente,muerte_cliente,mes_n,total_meses,producto_nuevo,...,cat2,cat3,brand,sku_size,stock_final,tn,plan_precios_cuidados,cust_request_qty,cust_request_tn,target
0,20001,10234,201701,201701,201912,201701,201912,1,17776,0,...,ROPA LAVADO,Liquido,ARIEL,3000.0,NaN,0.33579,0.0,1.0,0.33579,0.00000
1,20001,10032,201701,201701,201912,201701,201912,2,17776,0,...,ROPA LAVADO,Liquido,ARIEL,3000.0,NaN,12.31230,0.0,2.0,12.31230,15.39038
2,20001,10217,201701,201701,201912,201701,201912,3,17776,0,...,ROPA LAVADO,Liquido,ARIEL,3000.0,NaN,0.00000,NaN,NaN,NaN,0.00000
3,20001,10125,201701,201701,201912,201701,201911,4,17776,0,...,ROPA LAVADO,Liquido,ARIEL,3000.0,NaN,0.08954,0.0,1.0,0.08954,0.10074
4,20001,10012,201701,201701,201912,201701,201912,5,17776,0,...,ROPA LAVADO,Liquido,ARIEL,3000.0,NaN,6.97324,0.0,5.0,6.97324,195.14996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15487660,21281,10212,201708,201702,201708,201708,201912,3500,3504,0,...,NaN,NaN,NaN,NaN,NaN,0.00000,NaN,NaN,NaN,NaN
15487661,21281,10376,201708,201702,201708,201708,201912,3501,3504,0,...,NaN,NaN,NaN,NaN,NaN,0.00000,NaN,NaN,NaN,NaN
15487662,21281,10576,201708,201702,201708,201708,201911,3502,3504,0,...,NaN,NaN,NaN,NaN,NaN,0.00000,NaN,NaN,NaN,NaN
15487663,21281,10207,201708,201702,201708,201708,201912,3503,3504,0,...,NaN,NaN,NaN,NaN,NaN,0.00000,NaN,NaN,NaN,NaN


In [3]:
cat_cols = df.select_dtypes(include=['object']).columns.tolist()
cat_cols

['cat1', 'cat2', 'cat3', 'brand']

In [4]:
##### aplicamos OHE
df = preprocesamiento_pcp.aplicarOHE(df)
df

MemoryError: Unable to allocate 17.0 GiB for an array with shape (147, 15487665) and data type float64

In [ ]:
###### QUE PASARA SI A STOCK FINAL LE AÑADO LO QUE VENDIO? EXPERIMENTAR
df['stock_final'] = df['stock_final'].fillna(df['tn'])

In [ ]:
#### Estandarizar zscore
numericas = ['tn', 'stock_final', 'cust_request_qty', 'cust_request_tn', 'target']


if ESTOY_EN_KAGGLE:
    for col in numericas:
        df = preprocesamiento.normalizar_con_zscore(df, col, 201912) ##### Para kaggle reemplazar por 201912
    df.drop(columns=['target'], inplace=True)
    df.rename(columns={'target_zscore': 'target'}, inplace=True)
else:
    for col in numericas:
        df = preprocesamiento.normalizar_con_periodo201909(df, col) ##### uso media y desvio de periodo 201701 a 201909
    df.drop(columns=['target'], inplace=True)
    df.rename(columns={'target_zscore': 'target'}, inplace=True)


In [6]:
#### Tranformar: no eliminamos las variables... que lgb haga su magia...
numericas = ['tn', 'stock_final', 'cust_request_qty', 'cust_request_tn']
for col in numericas:
    df = preprocesamiento.transformar_logaritmca(df, col)
    

c:\Users\Usuario\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [7]:
df = preprocesamiento.eliminar_columnas_calculadas(df)

In [8]:
df

,product_id,periodo,nacimiento_producto,muerte_producto,mes_n,total_meses,producto_nuevo,ciclo_de_vida_inicial,sku_size,stock_final,...,brand_nan,tn_zscore,stock_final_zscore,cust_request_qty_zscore,cust_request_tn_zscore,target,tn_log,stock_final_log,cust_request_qty_log,cust_request_tn_log
0,20001,201701,201701,201912,1,36,0,0,3000.0,NaN,...,0.0,-1.576908,NaN,0.461891,-1.704215,-0.465448,6.841372,NaN,6.173786,6.844525
1,20001,201702,201701,201912,2,36,0,0,3000.0,NaN,...,0.0,-2.042103,NaN,-0.046911,-2.039835,-1.325726,6.683381,NaN,6.070738,6.727099
2,20001,201703,201701,201912,3,36,0,0,3000.0,NaN,...,0.0,-0.323111,NaN,0.786658,-0.435961,0.267471,7.173466,NaN,6.234411,7.194247
3,20001,201704,201701,201912,4,36,0,0,3000.0,NaN,...,0.0,-1.117043,NaN,-1.703222,-1.074260,0.333316,6.976312,NaN,5.634790,7.033457
4,20001,201705,201701,201912,5,36,0,0,3000.0,NaN,...,0.0,0.353285,NaN,2.865166,0.273781,-1.470536,7.315352,NaN,6.553933,7.347100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31357,21281,201704,201702,201708,3,7,1,1,NaN,NaN,...,1.0,-0.670410,NaN,NaN,NaN,1.850294,0.000000,NaN,NaN,NaN
31358,21281,201705,201702,201708,4,7,1,0,NaN,NaN,...,1.0,-0.670410,NaN,NaN,NaN,-0.707610,0.000000,NaN,NaN,NaN
31359,21281,201706,201702,201708,5,7,1,0,NaN,NaN,...,1.0,2.237248,NaN,1.414214,1.397345,0.272537,0.087406,NaN,2.197225,0.100198
31360,21281,201707,201702,201708,6,7,1,0,NaN,NaN,...,1.0,-0.670410,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN


In [10]:
if ESTOY_EN_KAGGLE:
    df.to_csv("./datasets/periodo_x_producto_con_target_transformado_201912.csv", sep=',', encoding='utf-8', index=False)
else:
    df.to_csv("./datasets/periodo_x_producto_con_target_transformado.csv", sep=',', encoding='utf-8', index=False) 

Experimentar: probar si conviene primero transformar y luego estandarizar.